In [106]:
import pandas as pd
import numpy as np
from rake_nltk import Rake
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer
import ast
from scipy.sparse import hstack

In [107]:
movies = pd.read_csv("../../Data/serendipity-sac2018/movies.csv", error_bad_lines=False)
imdbData = pd.read_csv("../Intermediate_data/IMDb_cldata.csv")
# movies = movies.drop(['imdbId', 'tmdbId', 'releaseDate'], axis = 1)
movies.head()

Skipping line 19833: expected 8 fields, saw 10
Skipping line 34143: expected 8 fields, saw 9
Skipping line 36015: expected 8 fields, saw 10
Skipping line 37260: expected 8 fields, saw 12
Skipping line 44379: expected 8 fields, saw 10
Skipping line 47551: expected 8 fields, saw 10



,movieId,title,releaseDate,directedBy,starring,imdbId,tmdbId,genres
0,1,Toy Story (1995),1995-11-19,John Lasseter,"Tim Allen, Tom Hanks, Don Rickles, Jim Varney,...",114709,862.0,"Adventure,Animation,Children,Comedy,Fantasy"
1,2,Jumanji (1995),1995-12-15,Joe Johnston,"Jonathan Hyde, Bradley Pierce, Robin Williams,...",113497,8844.0,"Adventure,Children,Fantasy"
2,3,Grumpier Old Men (1995),1995-01-01,Howard Deutch,"Jack Lemmon, Walter Matthau, Ann-Margret , Sop...",113228,15602.0,"Comedy,Romance"
3,4,Waiting to Exhale (1995),1996-01-15,Forest Whitaker,"Angela Bassett, Loretta Devine, Whitney Housto...",114885,31357.0,"Comedy,Drama,Romance"
4,5,Father of the Bride Part II (1995),1995-12-08,Charles Shyer,"Steve Martin, Martin Short, Diane Keaton, Kimb...",113041,11862.0,Comedy


In [108]:
imdbData['plot'] = imdbData['plot'].apply(lambda x: [  str(s.encode('utf-8')) for s in ast.literal_eval(x)])

In [109]:
imdbDataCols = ['plot', 'country codes', 'language codes', 'rating', 'runtimes', 'votes', 'imdbId']
imdbData[imdbDataCols].head()

,plot,country codes,language codes,rating,runtimes,votes,imdbId
0,[A cowboy doll is profoundly threatened and je...,['us'],['en'],8.3,81,837115.0,114709
1,[When two kids find and play a magical board g...,['us'],"['en', 'fr']",7.0,104,287007.0,113497
2,[John and Max resolve to save their beloved ba...,['us'],"['en', 'it', 'de']",6.7,101,23107.0,113228
3,"[Based on Terry McMillan's novel, this film fo...",['us'],['en'],5.9,124,9034.0,114885
4,[George Banks must deal not only with the preg...,['us'],['en'],6.0,106,32568.0,113041


In [110]:
movies = movies.merge(imdbData[imdbDataCols], on="imdbId")

In [111]:
movies.head()

,movieId,title,releaseDate,directedBy,starring,imdbId,tmdbId,genres,plot,country codes,language codes,rating,runtimes,votes
0,1,Toy Story (1995),1995-11-19,John Lasseter,"Tim Allen, Tom Hanks, Don Rickles, Jim Varney,...",114709,862.0,"Adventure,Animation,Children,Comedy,Fantasy",[A cowboy doll is profoundly threatened and je...,['us'],['en'],8.3,81,837115.0
1,2,Jumanji (1995),1995-12-15,Joe Johnston,"Jonathan Hyde, Bradley Pierce, Robin Williams,...",113497,8844.0,"Adventure,Children,Fantasy",[When two kids find and play a magical board g...,['us'],"['en', 'fr']",7.0,104,287007.0
2,3,Grumpier Old Men (1995),1995-01-01,Howard Deutch,"Jack Lemmon, Walter Matthau, Ann-Margret , Sop...",113228,15602.0,"Comedy,Romance",[John and Max resolve to save their beloved ba...,['us'],"['en', 'it', 'de']",6.7,101,23107.0
3,4,Waiting to Exhale (1995),1996-01-15,Forest Whitaker,"Angela Bassett, Loretta Devine, Whitney Housto...",114885,31357.0,"Comedy,Drama,Romance","[Based on Terry McMillan's novel, this film fo...",['us'],['en'],5.9,124,9034.0
4,5,Father of the Bride Part II (1995),1995-12-08,Charles Shyer,"Steve Martin, Martin Short, Diane Keaton, Kimb...",113041,11862.0,Comedy,[George Banks must deal not only with the preg...,['us'],['en'],6.0,106,32568.0


In [112]:
# discarding the commas between the actors' full names and getting only the first three names
movies['starring'] = movies['starring'].map(lambda x: x.split(',')[:3] if type(x) != float else np.nan)

# putting the genres in a list of words
movies['genres'] = movies['genres'].map(lambda x: x.lower().split(',') if type(x) != float else np.nan)

movies['directedBy'] = movies['directedBy'].map(lambda x: x.split(' ') if type(x) != float else np.nan)

movies['language codes'] = movies['language codes'].map(lambda x: ast.literal_eval(x) if type(x) != float else np.nan)
movies['country codes'] = movies['country codes'].map(lambda x: ast.literal_eval(x) if type(x) != float else np.nan)


# merging together first and last name for each actor and director, so it's considered as one word 
# and there is no mix up between people sharing a first name
for index, row in movies.iterrows():
    if type(row["starring"]) != float:
        row['starring'] = [x.lower().replace(' ','') for x in row['starring']]
    if type(row["directedBy"]) != float:
        row['directedBy'] = ''.join(row['directedBy']).lower()
    
movies.head()

,movieId,title,releaseDate,directedBy,starring,imdbId,tmdbId,genres,plot,country codes,language codes,rating,runtimes,votes
0,1,Toy Story (1995),1995-11-19,"[John, Lasseter]","[Tim Allen, Tom Hanks, Don Rickles]",114709,862.0,"[adventure, animation, children, comedy, fantasy]",[A cowboy doll is profoundly threatened and je...,[us],[en],8.3,81,837115.0
1,2,Jumanji (1995),1995-12-15,"[Joe, Johnston]","[Jonathan Hyde, Bradley Pierce, Robin Williams]",113497,8844.0,"[adventure, children, fantasy]",[When two kids find and play a magical board g...,[us],"[en, fr]",7.0,104,287007.0
2,3,Grumpier Old Men (1995),1995-01-01,"[Howard, Deutch]","[Jack Lemmon, Walter Matthau, Ann-Margret ]",113228,15602.0,"[comedy, romance]",[John and Max resolve to save their beloved ba...,[us],"[en, it, de]",6.7,101,23107.0
3,4,Waiting to Exhale (1995),1996-01-15,"[Forest, Whitaker]","[Angela Bassett, Loretta Devine, Whitney Hou...",114885,31357.0,"[comedy, drama, romance]","[Based on Terry McMillan's novel, this film fo...",[us],[en],5.9,124,9034.0
4,5,Father of the Bride Part II (1995),1995-12-08,"[Charles, Shyer]","[Steve Martin, Martin Short, Diane Keaton]",113041,11862.0,[comedy],[George Banks must deal not only with the preg...,[us],[en],6.0,106,32568.0


In [113]:
movies['Key_words'] = ""
for index, row in movies.iterrows():
    try:
        plot = row['plot'][0].decode('utf-8')

        # instantiating Rake, by default is uses english stopwords from NLTK
        # and discard all puntuation characters
        r = Rake()

        # extracting the words by passing the text
        r.extract_keywords_from_text(plot)

        # getting the dictionary whith key words and their scores
        key_words_dict_scores = r.get_word_degrees()

        # assigning the key words to the new column
        movies.at[index, 'Key_words'] = list(key_words_dict_scores.keys())
    except UnicodeDecodeError:
        print(index)
        break



In [114]:
# movies.drop('plot', inplace = True)
movies.head()

,movieId,title,releaseDate,directedBy,starring,imdbId,tmdbId,genres,plot,country codes,language codes,rating,runtimes,votes,Key_words
0,1,Toy Story (1995),1995-11-19,"[John, Lasseter]","[Tim Allen, Tom Hanks, Don Rickles]",114709,862.0,"[adventure, animation, children, comedy, fantasy]",[A cowboy doll is profoundly threatened and je...,[us],[en],8.3,81,837115.0,"[boy, toy, room, spaceman, doll, top, cowboy, ..."
1,2,Jumanji (1995),1995-12-15,"[Joe, Johnston]","[Jonathan Hyde, Bradley Pierce, Robin Williams]",113497,8844.0,"[adventure, children, fantasy]",[When two kids find and play a magical board g...,[us],"[en, fr]",7.0,104,287007.0,"[play, kids, kenneth, trapped, --, two, magica..."
2,3,Grumpier Old Men (1995),1995-01-01,"[Howard, Deutch]","[Jack Lemmon, Walter Matthau, Ann-Margret ]",113228,15602.0,"[comedy, romance]",[John and Max resolve to save their beloved ba...,[us],"[en, it, de]",6.7,101,23107.0,"[shop, catches, resolve, turning, beloved, res..."
3,4,Waiting to Exhale (1995),1996-01-15,"[Forest, Whitaker]","[Angela Bassett, Loretta Devine, Whitney Hou...",114885,31357.0,"[comedy, drama, romance]","[Based on Terry McMillan's novel, this film fo...",[us],[en],5.9,124,9034.0,"[relationships, different, novel, based, anony..."
4,5,Father of the Bride Part II (1995),1995-12-08,"[Charles, Shyer]","[Steve Martin, Martin Short, Diane Keaton]",113041,11862.0,[comedy],[George Banks must deal not only with the preg...,[us],[en],6.0,106,32568.0,"[unexpected, daughter, deal, wife, also, cupca..."


In [115]:
movies.drop('movieId', inplace=True, axis =1)
movies.set_index('title', inplace = True)
movies['bag_of_words'] = ''
bagOfWordColumns = ["directedBy", "starring", "genres", "country codes", "language codes", "Key_words"]
errorIndices = []
for index, row in movies.iterrows():
    words = ''
    for col in bagOfWordColumns:
        try:
            if type(row[col]) != float:
                words = words + ' '.join(row[col])+ ' '
        except UnicodeDecodeError:
            errorIndices.append(index)
    try:
        movies.at[index, 'bag_of_words'] = words
    except:
        print(index)
    
# movies.drop(columns = [col for col in movies.columns if col!= 'bag_of_words'], inplace = True)
movies.head()

nan


,releaseDate,directedBy,starring,imdbId,tmdbId,genres,plot,country codes,language codes,rating,runtimes,votes,Key_words,bag_of_words
title,,,,,,,,,,,,,,
Toy Story (1995),1995-11-19,"[John, Lasseter]","[Tim Allen, Tom Hanks, Don Rickles]",114709,862.0,"[adventure, animation, children, comedy, fantasy]",[A cowboy doll is profoundly threatened and je...,[us],[en],8.3,81,837115.0,"[boy, toy, room, spaceman, doll, top, cowboy, ...",John Lasseter Tim Allen Tom Hanks Don Rickle...
Jumanji (1995),1995-12-15,"[Joe, Johnston]","[Jonathan Hyde, Bradley Pierce, Robin Williams]",113497,8844.0,"[adventure, children, fantasy]",[When two kids find and play a magical board g...,[us],"[en, fr]",7.0,104,287007.0,"[play, kids, kenneth, trapped, --, two, magica...",Joe Johnston Jonathan Hyde Bradley Pierce Ro...
Grumpier Old Men (1995),1995-01-01,"[Howard, Deutch]","[Jack Lemmon, Walter Matthau, Ann-Margret ]",113228,15602.0,"[comedy, romance]",[John and Max resolve to save their beloved ba...,[us],"[en, it, de]",6.7,101,23107.0,"[shop, catches, resolve, turning, beloved, res...",Howard Deutch Jack Lemmon Walter Matthau Ann...
Waiting to Exhale (1995),1996-01-15,"[Forest, Whitaker]","[Angela Bassett, Loretta Devine, Whitney Hou...",114885,31357.0,"[comedy, drama, romance]","[Based on Terry McMillan's novel, this film fo...",[us],[en],5.9,124,9034.0,"[relationships, different, novel, based, anony...",Forest Whitaker Angela Bassett Loretta Devine...
Father of the Bride Part II (1995),1995-12-08,"[Charles, Shyer]","[Steve Martin, Martin Short, Diane Keaton]",113041,11862.0,[comedy],[George Banks must deal not only with the preg...,[us],[en],6.0,106,32568.0,"[unexpected, daughter, deal, wife, also, cupca...",Charles Shyer Steve Martin Martin Short Dian...


In [121]:
# instantiating and generating the count matrix
count = CountVectorizer()
count_matrix = count.fit_transform(movies['bag_of_words'])
print(count_matrix.shape)
# count_matrix = hstack((count_matrix,np.array(movies['runtimes'])[:,None]))
# count_matrix = hstack((count_matrix,np.array(movies['rating'])[:,None]))
# count_matrix = hstack((count_matrix,np.array(movies['votes'])[:,None]))
print(count_matrix.shape)
# creating a Series for the movie titles so they are associated to an ordered numerical
# list I will use later to match the indexes
indices = pd.Series(movies.index)
indices[:5]

(43947, 89171)
(43947, 89171)


0                      Toy Story (1995)
1                        Jumanji (1995)
2               Grumpier Old Men (1995)
3              Waiting to Exhale (1995)
4    Father of the Bride Part II (1995)
Name: title, dtype: object

(43947, 89175)


In [122]:
cosine_sim = cosine_similarity(count_matrix, count_matrix)

In [123]:
def recommendations(title, cosine_sim, movies):
    indices = pd.Series(movies.index)
    recommended_movies = []
    
    # gettin the index of the movie that matches the title
    idx = indices[indices == title].index[0]

    # creating a Series with the similarity scores in descending order
    score_series = pd.Series(cosine_sim[idx]).sort_values(ascending = False)

    # getting the indexes of the 10 most similar movies
    top_10_indexes = list(score_series.iloc[1:11].index)
    
    # populating the list with the titles of the best 10 matching movies
    for i in top_10_indexes:
        recommended_movies.append(list(movies.index)[i])
        
    return recommended_movies

In [124]:
recommendations("Toy Story (1995)", cosine_sim, movies)

['Toy Story Toons: Small Fry (2011)',
 'Toy Story 2 (1999)',
 'Tin Toy (1988)',
 'Toy Story Toons: Partysaurus Rex (2012)',
 'Toy Story Toons: Hawaiian Vacation (2011)',
 'Buzz Lightyear of Star Command: The Adventure Begins (2000)',
 'Toy Story That Time Forgot (2014)',
 'Big (1988)',
 'Toy Story 3 (2010)',
 'Education for Death (1943)']

In [146]:
recommendations("Argo (2012)", cosine_sim, movies)

['Town, The (2010)',
 'Good Will Hunting (1997)',
 'Live by Night (2017)',
 'Gone Baby Gone (2007)',
 '200 Cigarettes (1999)',
 'The Finest Hours (2016)',
 'Man About Town (2006)',
 'State of Play (2009)',
 'To Live and Die in L.A. (1985)',
 'Escape from Sobibor (1987)']

In [145]:
for i in indices:
    try:
        if "Argo" in i:
            print(i)
    except:
        print(i)

nan
Jason and the Argonauts (1963)
Jason and the Argonauts (2000)
Argo
Argo (2004)
Argo (2012)
Argo 2 (2015)


['Toy Story (1995)',
 'Jumanji (1995)',
 'Grumpier Old Men (1995)',
 'Waiting to Exhale (1995)',
 'Father of the Bride Part II (1995)',
 'Heat (1995)',
 'Sabrina (1995)',
 'Tom and Huck (1995)',
 'Sudden Death (1995)',
 'GoldenEye (1995)',
 'American President, The (1995)',
 'Dracula: Dead and Loving It (1995)',
 'Balto (1995)',
 'Nixon (1995)',
 'Cutthroat Island (1995)',
 'Casino (1995)',
 'Sense and Sensibility (1995)',
 'Four Rooms (1995)',
 'Ace Ventura: When Nature Calls (1995)',
 'Money Train (1995)',
 'Get Shorty (1995)',
 'Copycat (1995)',
 'Assassins (1995)',
 'Powder (1995)',
 'Leaving Las Vegas (1995)',
 'Othello (1995)',
 'Now and Then (1995)',
 'Persuasion (1995)',
 'City of Lost Children, The (Cit\xc3\xa9 des enfants perdus, La) (1995)',
 'Shanghai Triad (Yao a yao yao dao waipo qiao) (1995)',
 'Dangerous Minds (1995)',
 'Twelve Monkeys (a.k.a. 12 Monkeys) (1995)',
 'Wings of Courage (1995)',
 'Babe (1995)',
 'Carrington (1995)',
 'Dead Man Walking (1995)',
 'Across the 